# Mục tiêu:
* Hiểu rõ cách hoạt động của kỹ thuật giấu tin LSB (Least Significant Bit) trong thực tế.

* Tự tay viết mã Python để thực hiện quá trình giấu tin (encode) vào một tệp âm thanh WAV.

* Tự tay viết mã Python để tách tin (decode) từ tệp âm thanh đã được giấu tin.

* **Đánh** giá tính bất khả tri của phương pháp bằng cách so sánh âm thanh gốc và âm thanh đã giấu tin.

# Bước 1: Chuẩn bị Môi trường và Dữ liệu
Đầu tiên, chúng ta cần một tệp âm thanh WAV để làm môi trường chứa (cover-medium). Thay vì tải lên một tệp, chúng ta sẽ tự tạo ra một tệp âm thanh đơn giản chứa một nốt nhạc (sóng sin) bằng Python.

In [1]:
import wave
import math
import struct

# --- Các thông số cho tệp âm thanh ---
sample_rate = 44100.0  # Tần số lấy mẫu (Hz)
duration = 5           # Độ dài âm thanh (giây)
frequency = 440.0      # Tần số của nốt nhạc (A4)
output_file = "original_audio.wav"

# --- Tạo tệp âm thanh WAV ---
wav_file = wave.open(output_file, 'w')

# Thiết lập các thông số  WAV
num_channels = 1      # 1 kênh (mono)
samp_width = 2        # 2 bytes/mẫu (16-bit)
num_frames = int(duration * sample_rate)
comptype = "NONE"
compname = "not compressed"

wav_file.setparams((num_channels, samp_width, int(sample_rate), num_frames, comptype, compname))

# Ghi dữ liệu sóng sin vào tệp
for i in range(num_frames):
    # Tính giá trị mẫu
    value = int(32767.0 * math.sin(2.0 * math.pi * frequency * i / sample_rate))
    # Đóng gói giá trị thành 2 bytes và ghi vào tệp
    data = struct.pack('<h', value)
    wav_file.writeframes(data)

wav_file.close()
print(f"Đã tạo thành công tệp '{output_file}'")

# --- Nghe thử tệp âm thanh ---
from IPython.display import Audio
print("Âm thanh gốc:")
Audio(output_file)

Đã tạo thành công tệp 'original_audio.wav'
Âm thanh gốc:


# Bước 2: Xây dựng Hàm Giấu Tin (Encode)
Bây giờ, chúng ta sẽ viết một hàm Python để thực hiện quá trình giấu một thông điệp văn bản vào tệp original_audio.wav bằng kỹ thuật LSB.

Nguyên tắc:

* Đọc dữ liệu âm thanh dưới dạng một chuỗi các byte.

* Chuyển thông điệp bí mật thành một chuỗi bit nhị phân.

* Thêm một "dấu hiệu kết thúc" đặc biệt vào cuối thông điệp để biết khi nào cần dừng quá trình giải mã.

* Lặp qua từng byte của dữ liệu âm thanh, thay thế bit cuối cùng (LSB) của nó bằng một bit từ thông điệp bí mật.

* Lưu dữ liệu âm thanh đã bị thay đổi vào một tệp WAV mới.



In [2]:
def encode_audio(audio_path, secret_message, output_path):
    """
    Giấu một thông điệp vào tệp âm thanh WAV bằng kỹ thuật LSB.
    """
    # Mở tệp âm thanh gốc để đọc
    wav = wave.open(audio_path, mode='rb')
    # Đọc toàn bộ dữ liệu âm thanh dưới dạng byte
    frame_bytes = bytearray(list(wav.readframes(wav.getnframes())))

    # Thêm dấu hiệu kết thúc vào thông điệp bí mật
    secret_message += '#####'

    # Chuyển thông điệp thành chuỗi bit
    bits = ''.join([format(ord(char), '08b') for char in secret_message])

    # Kiểm tra xem tệp âm thanh có đủ lớn để chứa thông điệp không
    if len(bits) > len(frame_bytes):
        raise ValueError("Thông điệp quá dài để giấu trong tệp âm thanh này!")

    # Bắt đầu quá trình giấu tin
    for i, bit in enumerate(bits):
        # Xóa bit LSB của byte hiện tại
        frame_bytes[i] = (frame_bytes[i] & 254)
        # Ghi bit của thông điệp vào vị trí LSB
        frame_bytes[i] = (frame_bytes[i] | int(bit))

    # Chuyển đổi bytearray trở lại dạng bytes
    frame_modified = bytes(frame_bytes)

    # Ghi dữ liệu đã sửa đổi vào một tệp WAV mới
    with wave.open(output_path, 'wb') as fd:
        fd.setparams(wav.getparams())
        fd.writeframes(frame_modified)

    wav.close()
    print(f"Đã giấu thông điệp thành công vào tệp '{output_path}'")

# Bước 3: Xây dựng Hàm Tách Tin (Decode)
Tiếp theo, chúng ta cần một hàm để làm điều ngược lại: đọc một tệp âm thanh đã giấu tin và trích xuất thông điệp bí mật.

Nguyên tắc:

* Đọc dữ liệu từ tệp âm thanh đã giấu tin (stego-audio).

* Lặp qua từng byte, trích xuất bit cuối cùng (LSB).

* Ghép các bit LSB lại với nhau thành chuỗi nhị phân.

* Cứ mỗi 8 bit, chuyển đổi chúng thành một ký tự.

* Dừng lại khi gặp "dấu hiệu kết thúc" mà chúng ta đã định nghĩa.

In [3]:
def decode_audio(audio_path):
    """
    Tách thông điệp bí mật từ một tệp âm thanh WAV.
    """
    # Mở tệp âm thanh đã giấu tin
    wav = wave.open(audio_path, mode='rb')
    # Đọc dữ liệu âm thanh
    frame_bytes = bytearray(list(wav.readframes(wav.getnframes())))

    # Trích xuất các bit LSB từ mỗi byte
    extracted_bits = [str(frame_byte & 1) for frame_byte in frame_bytes]
    extracted_bits_str = "".join(extracted_bits)

    # Chia chuỗi bit thành các khối 8-bit (tương ứng với các ký tự)
    chars = []
    for i in range(0, len(extracted_bits_str), 8):
        byte = extracted_bits_str[i:i+8]
        if len(byte) == 8:
            # Chuyển đổi byte nhị phân thành ký tự
            chars.append(chr(int(byte, 2)))

    # Ghép các ký tự lại thành chuỗi
    message = "".join(chars)

    # Tìm và cắt bỏ dấu hiệu kết thúc
    if '#####' in message:
        return message.split('#####')
    else:
        return "Không tìm thấy thông điệp hoặc dấu hiệu kết thúc bị lỗi."

    wav.close()

# Bước 4: Thực Thi và Kiểm Tra Kết Quả
Chúng ta sẽ định nghĩa một thông điệp bí mật, giấu nó vào tệp âm thanh gốc, sau đó tách nó ra và so sánh kết quả.

In [4]:
# --- Định nghĩa các biến ---
original_file = "original_audio.wav"
stego_file = "stego_audio.wav"
secret_message_to_hide = "Day la mot thong diep bi mat duoc giau trong am thanh."

# --- Bắt đầu thực thi ---
print("Bắt đầu quá trình giấu tin...")
# 1. Giấu thông điệp
encode_audio(original_file, secret_message_to_hide, stego_file)

print("\nBắt đầu quá trình tách tin...")
# 2. Tách thông điệp
decoded_message = decode_audio(stego_file)

# --- In kết quả ---
print("-" * 30)
print(f"Thông điệp gốc: '{secret_message_to_hide}'")
print(f"Thông điệp được tách ra: '{decoded_message}'")
print("-" * 30)

# --- So sánh và nghe thử ---
if secret_message_to_hide == decoded_message:
    print("THÀNH CÔNG: Thông điệp gốc và thông điệp tách ra hoàn toàn trùng khớp!")
else:
    print("THẤT BẠI: Có lỗi trong quá trình giấu hoặc tách tin.")

print("\nBây giờ hãy nghe thử cả hai tệp âm thanh để so sánh:")
print("\n1. Âm thanh gốc:")
display(Audio(original_file))

print("\n2. Âm thanh đã giấu tin (Stego Audio):")
display(Audio(stego_file))

Bắt đầu quá trình giấu tin...
Đã giấu thông điệp thành công vào tệp 'stego_audio.wav'

Bắt đầu quá trình tách tin...
------------------------------
Thông điệp gốc: 'Day la mot thong diep bi mat duoc giau trong am thanh.'
Thông điệp được tách ra: '['Day la mot thong diep bi mat duoc giau trong am thanh.', 'ê\x06Ø\x00¨Ù.0\'Z¤\x11û\x82M\x80"\x0f\x13P´êu\x15\x0f²\x9e\x16\x08\x82\x19\x9dOQ@\x86¬.U\x91à\x08£OtéùU\x15\x12ç{?("\x1eR0\x8e\x86\nK~´1ú\n#Ä\x0b\x9f=·àÀ\x9a+\x87Úp¬â\x8d½¦\x03q\x89(r××`dÖbe?j\x1dÆ¦ÍU÷%¯ÀaV!fþSÄ\x15uò±_\x15H\x1fåº9|IÕWÁ6\x1b\x8c^¡©+\x9f$5\x7fÞ9¢H0_ÇgxÓøUWAØ\x84ð¹\t\n\x14È\x91\x8f\x7fU¤\x1c\x9dOÅþ\x18.«:\x1d]úÇû\t+¨ôI¸Û\x83wÕ\x8cý`+Ír\x1b{\x7f\x16\xadÕöL¼ôÃ\n}\x84åhê\x14\x1fçÀär\x8fYd¦¶L\xa0"ü\x1a\x8b\x8b\x1d\x82kÕÂ¦p(\x89{\'³D§ûÓ[¨o¢\x02\x1f\x9bþ×\x1e\xa0jB6\x14ð\n\x8aq7d\x1fSWµ¤w!H\x00\x07©\'\x88Èj\x05\x97Mó\x9e*\x80\x94Ú;Óñþ\x06{\x1c\x91h\x00#F\x83\x9f°ê\x17@\x1a\x01½\xa0\x80\x14i\x0e¬ÁÜ\x01ÅjðØ\xa0\x02lüb\x1fÊ\x90ÍÌ$ÅÈ(-\x87Óq\x89\\ò²}æõ\x8f({ß5eè²\x9a;Tiß_¡\x94¾\x


2. Âm thanh đã giấu tin (Stego Audio):


# Bài tập
* Thử thay đổi thông điệp bí mật với độ dài khác nhau.

* Thử sử dụng một tệp WAV khác của riêng bạn (bạn có thể tải tệp lên Google Colab) và xem kết quả

# Bài 1: Thử thay đổi **độ dài thông điệp bí mật**


In [5]:
import os
import wave

messageList = [
    "My secret message is hidden here",
    "Tôi đang làm việc tại thành phố Hồ Chí Minh",
    "I'm learning Python"
]

# 3) Kiểm tra lại file WAV gốc từ phần trước
audio_path = globals().get("output_file", "original_audio.wav")
if not os.path.exists(audio_path):
    raise FileNotFoundError(
        f"Khong tim thay {audio_path}. Hay chay cell tao WAV o phan truoc truoc khi thu nghiem."
    )

# In thử sức chứa ước lượng để biết giới hạn
with wave.open(audio_path, "rb") as w:
    nframes   = w.getnframes()
    sampwidth = w.getsampwidth()
    nchannels = w.getnchannels()
    total_bytes = nframes * sampwidth * nchannels

capacity_chars_est = (total_bytes // 8) - 5
print(f"Suc chua uoc luong ~ {capacity_chars_est} ky tu")
print("So luong bien the se thu:", len(messageList))


Suc chua uoc luong ~ 55120 ky tu
So luong bien the se thu: 3


In [6]:
import time
import pandas as pd

rows = []

for idx, secret_message_to_hide in enumerate(messageList):
    out_path = f"stego_msg_{idx}.wav"
    t0 = time.time()
    error = None
    decoded_msg = None
    success = False

    try:
        # Giấu tin vào bản sao WAV (không sửa file gốc)
        encode_audio(audio_path, secret_message_to_hide, out_path)

        # Tách tin
        result = decode_audio(out_path)
        if isinstance(result, list) and len(result) > 0:
            decoded_msg = result[0]
        elif isinstance(result, str):
            decoded_msg = result
        else:
            decoded_msg = None

        # Đánh giá: so sánh 100% hai chuỗi
        success = (decoded_msg == secret_message_to_hide)

    except Exception as e:
        error = str(e)

    dt_ms = (time.time() - t0) * 1000.0
    rows.append({
        "index": idx,
        "length_chars": len(secret_message_to_hide),
        "success_decode": success,
        "error": error,
        "stego_path": out_path if error is None else None,
        "preview_expected": secret_message_to_hide[:60] + ("..." if len(secret_message_to_hide) > 60 else ""),
        "preview_decoded": (decoded_msg[:60] + ("..." if (decoded_msg and len(decoded_msg) > 60) else "")) if decoded_msg else None,
        "time_ms": round(dt_ms, 2),
    })

Đã giấu thông điệp thành công vào tệp 'stego_msg_0.wav'
Đã giấu thông điệp thành công vào tệp 'stego_msg_1.wav'
Đã giấu thông điệp thành công vào tệp 'stego_msg_2.wav'


In [7]:
# KẾT QUẢ:
df_eval = pd.DataFrame(rows)
df_eval
print(f"Thong ke: so mau = {len(df_eval)}; so mau thanh cong = {df_eval['success_decode'].sum()}")
display(df_eval)


Thong ke: so mau = 3; so mau thanh cong = 2


,index,length_chars,success_decode,error,stego_path,preview_expected,preview_decoded,time_ms
0,0,32,True,None,stego_msg_0.wav,My secret message is hidden here,My secret message is hidden here,93.03
1,1,43,False,None,stego_msg_1.wav,Tôi đang làm việc tại thành phố Hồ Chí Minh,Không tìm thấy thông điệp hoặc dấu hiệu kết th...,77.45
2,2,19,True,None,stego_msg_2.wav,I'm learning Python,I'm learning Python,71.85


# Bài 2: Thử sử dụng một tệp WAV khác**

In [10]:
# Sử dụng một tệp WAV khác và gán thông điệp mẫu
# --- Định nghĩa các biến ---
original_file = "audio_data.wav"
stego_file = "audio_data.wav"
secret_message_to_hide = "Good day to you"

# --- Bắt đầu thực thi ---
print("Bắt đầu quá trình giấu tin...")
# 1. Giấu thông điệp
encode_audio(original_file, secret_message_to_hide, stego_file)

print("\nBắt đầu quá trình tách tin...")
# 2. Tách thông điệp
decoded_message = decode_audio(stego_file)

# --- In kết quả ---
print("-" * 30)
print(f"Thông điệp gốc: '{secret_message_to_hide}'")
print(f"Thông điệp được tách ra: '{decoded_message}'")
print("-" * 30)

# --- So sánh và nghe thử ---
if secret_message_to_hide == decoded_message:
    print("THÀNH CÔNG: Thông điệp gốc và thông điệp tách ra hoàn toàn trùng khớp!")
else:
    print("THẤT BẠI: Có lỗi trong quá trình giấu hoặc tách tin.")

print("\nBây giờ hãy nghe thử cả hai tệp âm thanh để so sánh:")
print("\n1. Âm thanh gốc:")
display(Audio(original_file))

print("\n2. Âm thanh đã giấu tin (Stego Audio):")
display(Audio(stego_file))


Bắt đầu quá trình giấu tin...
Đã giấu thông điệp thành công vào tệp 'audio_data.wav'

Bắt đầu quá trình tách tin...
------------------------------
Thông điệp gốc: 'Good day to you'
Thông điệp được tách ra: '['Good day to you', 'ª*\x82K.·uõUÐ0\x02*¿¨ªØ§|\x16²\x80\x8c#ôSÕ×}\xadTô\x7fuÛ*\x00\x8a¿ýßÜ8\x82\n\x02¥\x82¢n\x9d]ÿÿÊ\x80(\n"¡\x7fWÝWýÿz\x05\x14\x88\x82\x0f/ZØ\x82"¨\xa0\x0b\x8dßß\x1e7\x87\x7fò\x82\x08\x8a\xa05]UÝÝ\x7f_W(\x82*\x02âª"\x80\x8a&º\x88\x02£\x7f}}}ÿ\x1d]wÞç\x8a"\x104\x82·Ý}â\x8a\x80\x00((\n(¡\x7fUuwUõßõ\x7fu\x15(>\x00( \n\n\x03\\\x07W|]ØÄ\x8du_àÂ\x00\x08\x08\x0fIGÿ~\x02(\x02#w}Ûuÿu÷qJ\x00\x08¨ª\x82«â\x08 ¢\x8a\x00ámWõwý]ý)r_ÛcJ¢\x80&1b¢¢¢\x8a¸ßs_wW\x7fÕ}\x82\x08\n \x08!ß\x7f\x00\x00*\n\xa0\n\n\x9fÿß×ß_ÿUði\x03kMm]ôÜ\x10ý\xa0\x08\xa0\n\xa0\n¢\x08 wÝÕ\x7f-UýÙË_}@"0\n\x08ª\x00\x80(×ß\x15w}ý¢¨\x80\x08}u×WS0¢¨"\nU\x84¥²Ýr\x00\x88\x08\x86õU\x8f_ýv\x08\x00"\x96âOÜÕUð*bP®*"\x8a#\x7f#×\x08õU}w\x7f]Å-uó\x82\x80\xa0\n¢`\x95×[ÕiÀ_÷Ñ~Õ}RP\x88\x00(\x8a\x8a\x07}\x7f×&Oýþ\x98ü¨Ö]ò\x02\x80


2. Âm thanh đã giấu tin (Stego Audio):
